# London Air - Site information collection

# 0. Set-up

In [1]:
## import liabrairies
import pandas as pd
import numpy as np
import re
import requests
import json

# 1. Functions

In [2]:
def request_json(link):
    response = requests.get(link)
    rep_json = response.text
    parsed = json.loads(rep_json)
    return parsed

In [3]:
## Function to get info on sites

def get_info_sites(group_name):

    # Get json response for the group
    site_request = ('/Information/MonitoringSites/GroupName=' + group_name + '/Json')
    link_info = base_api + site_request
    info_json = request_json(link_info)
    info_json 
    
    # Create empty list to store info during parsing
    LocalAuthorityCode = []
    LocalAuthorityName = []
    SiteCode = []
    SiteName = []
    SiteType = []
    DateClosed = []
    DateOpened = []
    Latitude = []
    Longitude = []
    LatitudeWGS84 = []
    LongitudeWGS84 = []
    DisplayOffsetX = []
    DisplayOffsetY = []
    DataOwner = []
    DataManager = []
    SiteLink = []

    # Loop through the json file and append storing lists
    for i in range(len(info_json['Sites']['Site'])):
        root_data = info_json['Sites']['Site']
        LocalAuthorityCode.append(root_data[i]['@LocalAuthorityCode'])
        LocalAuthorityName.append(root_data[i]['@LocalAuthorityName'])

        SiteCode.append(root_data[i]['@SiteCode'])
        SiteName.append(root_data[i]['@SiteName'])
        SiteType.append(root_data[i]['@SiteType'])

        DateClosed.append(root_data[i]['@DateClosed'])
        DateOpened.append(root_data[i]['@DateOpened'])

        Latitude.append(root_data[i]['@Latitude'])
        Longitude.append(root_data[i]['@Longitude'])
        LatitudeWGS84.append(root_data[i]['@LatitudeWGS84'])
        LongitudeWGS84.append(root_data[i]['@LongitudeWGS84'])
        DisplayOffsetX.append(root_data[i]['@DisplayOffsetX'])
        DisplayOffsetY.append(root_data[i]['@DisplayOffsetY'])

        DataOwner.append(root_data[i]['@DataOwner'])
        DataManager.append(root_data[i]['@DataManager'])
        SiteLink.append(root_data[i]['@SiteLink'])

    # Create final dataframe
    df_info_sites = pd.DataFrame(np.column_stack([LocalAuthorityCode, LocalAuthorityName,
                                                SiteCode, SiteName, SiteType,
                                                DateClosed, DateOpened,
                                                Latitude, Longitude, LatitudeWGS84, LongitudeWGS84,
                                                DisplayOffsetX, DisplayOffsetY,
                                                DataOwner, DataManager, SiteLink]),

                                   columns=['LocalAuthorityCode', 'LocalAuthorityName',
                                            'SiteCode', 'SiteName', 'SiteType',
                                            'DateClosed', 'DateOpened',
                                            'Latitude', 'Longitude', 'LatitudeWGS84', 'LongitudeWGS84',
                                            'DisplayOffsetX', 'DisplayOffsetY',
                                            'DataOwner', 'DataManager', 'SiteLink'])
    
    # Remove sites that have been closed
    df_info_sites = df_info_sites[df_info_sites['DateClosed'] == '']
    df_info_sites = df_info_sites.reset_index(drop = True)
    
    return df_info_sites

# 2. Execution

In [4]:
base_api = 'https://api.erg.ic.ac.uk/AirQuality'

In [5]:
# Execute API request for the site group 'London'
df_sites = get_info_sites('London')

In [6]:
# Formatting of results for input into PostgreSQL schema
col_list = ['SiteCode', 'SiteName', 'Latitude', 'Longitude']
df_sites_tosql = df_sites[col_list]

df_sites_tosql.columns = ['site_code', 'site_name', ' latitude', 'longitude']

# 3. Output

In [7]:
df_sites_tosql

,site_code,site_name,latitude,longitude
0,BG1,Barking and Dagenham - Rush Green,51.563752,0.177891
1,BG2,Barking and Dagenham - Scrattons Farm,51.529389,0.132857
2,BX2,Bexley - Belvedere,51.4906102082147,0.158914493927518
3,BX0,Bexley - Belvedere FDMS,51.4906102082147,0.158914493927518
4,BQ7,Bexley - Belvedere West,51.4946486813055,0.137279111232178
...,...,...,...,...
104,MY1,Westminster - Marylebone Road,51.52254,-0.15459
105,MY7,Westminster - Marylebone Road FDMS,51.52254,-0.15459
106,WM6,Westminster - Oxford Street,51.5139287404213,-0.152792701881935
107,WMB,Westminster - Oxford Street East,51.516066,-0.13516388
